## R7 Arm Mug Grasping on Table w/ Updating Mug Position (Zero-Order PBDS)

In [1]:
using PBDS, StaticArrays, LinearAlgebra, GeometryBasics, Rotations, StaticNumbers, CoordinateTransformations, GeometryTypes, ForwardDiff, ComputedFieldTypes
import ColorTypes: RGBA
using RigidBodyDynamics, MeshCatMechanisms, MeshCat
using BenchmarkTools

GBVec = GeometryBasics.Vec

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17923921800408697935\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-18232879251434313589\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17923921800408697935\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

GeometryBasics.Vec

## Setup

## Damping

### Baseline Joint Damping

In [2]:
function PBDS.dissipative_forces_chart(xn, vn, task::Damping{<:Identity{R7,R7,S}}, CN::Chart{1,R7}) where S
    -1*SA[25, 25, 18, 18, 16, 14, 18].*vn
end

PBDS.metric_chart(xn, task::Damping{<:Identity{R7,R7,S}}, CN::Chart{1,R7}) where S = PBDS.default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Damping{<:Identity{R7,R7,S}}, CN::Chart{1,R7}) where S = 0.
PBDS.weight_metric_chart(xn, vn, task::Damping{<:Identity{R7,R7,S}}, CN::Chart{1,R7}) where S = PBDS.default_weight_metric(xn, vn, task, CN)

### End Effector Position Damping

In [3]:
function PBDS.dissipative_forces_chart(xn, vn, task::Damping{<:Identity{R3,R3,S}}, CN::Chart{1,R3}) where S
    -40*vn
end

PBDS.metric_chart(xn, task::Damping{<:Identity{R3,R3,S}}, CN::Chart{1,R3}) where S = PBDS.default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Damping{<:Identity{R3,R3,S}}, CN::Chart{1,R3}) where S = 0.
PBDS.weight_metric_chart(xn, vn, task::Damping{<:Identity{R3,R3,S}}, CN::Chart{1,R3}) where S = PBDS.default_weight_metric(xn, vn, task, CN)

### S2 End Effector Damping About Mug

In [4]:
PBDS.default_coord_rep(::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = 0.
function PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S
    -1*vne
end
PBDS.weight_metric_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = 
    default_weight_metric(xne, vne, task)

PBDS.home_task_chart(task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = Chart{SterProjSouth,S2}()

## Constraints

### Joint Limit Task

In [5]:
struct JointLimit{F,S} <: PBDS.Task{F}
    task_map::F
    hi::S
    lo::S
end
function PBDS.metric_chart(xn, task::JointLimit{<:Coordinate{R7,R1,S},S}, CN::Chart{1,R1}) where S
    ψx_hi = exp(1.e1 / (task.hi-xn[1])^2)
    ψx_lo = exp(1.e1 / (xn[1]-task.lo)^2)
    G = SMatrix{1,1,eltype(xn)}([ψx_hi + ψx_lo])
end

function PBDS.weight_metric_chart(xn, vn, task::JointLimit{<:Coordinate{R7,R1,S},S}, CN::Chart{1,R1}) where S
    offset_distance = π/4
    λ = ((task.hi - xn[1] < offset_distance) && (vn[1] > 0.)) || ((xn[1] - task.lo < offset_distance) && (vn[1] < 0.)) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

PBDS.potential_chart(xn, task::JointLimit{<:Coordinate{R7,R1,S},S}, CN::Chart{1,R1}) where S = 0.
PBDS.dissipative_forces_chart(xn, vn, task::JointLimit{<:Coordinate{R7,R1,S},S}, CN::Chart{1,R1}) where S = 0*vn

### Sphere-Sphere Self-Collision Avoidance

In [6]:
@ext_standard_task_type SelfAvoidance

function PBDS.metric_chart(xn, task::SelfAvoidance{<:LinkSpherePairDistance{R7,R1,S}}, CN::Chart{1,R1}) where S
    ψx = exp(1.e0 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([ψx])
end

function PBDS.active_weight_position_chart(xn, task::SelfAvoidance{<:LinkSpherePairDistance{R7,R1,S}}, CN::Chart{1,R1}) where S
    offset_distance = 0.1
    xn[1] < offset_distance
end

function PBDS.weight_metric_chart(xn, vn, task::SelfAvoidance{<:LinkSpherePairDistance{R7,R1,S}}, CN::Chart{1,R1}) where S
    λ = (PBDS.active_weight_position_chart(xn, task, CN) && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

PBDS.potential_chart(xn, task::SelfAvoidance{<:LinkSpherePairDistance{R7,R1,S}}, CN::Chart{1,R1}) where S = 0.
function PBDS.dissipative_forces_chart(xn, vn, task::SelfAvoidance{<:LinkSpherePairDistance{R7,R1,S}}, CN::Chart{1,R1}) where S
    -0.0*vn
end

### Cup Avoidance

In [7]:
function PBDS.metric_chart(xn, task::CollisionAvoidance{<:DistanceSphereToCupDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S
    ψx = exp(1.e-2 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([1. + ψx])
end

function PBDS.active_weight_position_chart(xn, task::CollisionAvoidance{<:DistanceSphereToCupDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S
    offset_distance = 0.1
    xn[1] < offset_distance
end

function PBDS.weight_metric_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToCupDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S
    λ = (PBDS.active_weight_position_chart(xn, task, CN) && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

PBDS.potential_chart(xn, task::CollisionAvoidance{<:DistanceSphereToCupDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S = 0.
PBDS.dissipative_forces_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToCupDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S = 0*vn

### Box Avoidance

In [8]:
function PBDS.metric_chart(xn, task::CollisionAvoidance{<:DistanceSphereToBoxDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S
    ψx = exp(1.e-3 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([1. + ψx])
end

function PBDS.active_weight_position_chart(xn, task::CollisionAvoidance{<:DistanceSphereToBoxDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S
    offset_distance = 0.4
    xn[1] < offset_distance
end

function PBDS.weight_metric_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToBoxDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S
    λ = (PBDS.active_weight_position_chart(xn, task, CN) && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

PBDS.potential_chart(xn, task::CollisionAvoidance{<:DistanceSphereToBoxDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S = 0.
PBDS.dissipative_forces_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToBoxDynamic{R3,R1,S}}, CN::Chart{1,R1}) where S = 0*vn

## Attractors

### Toggled Distance to Cylinder Above Cup

In [9]:
struct ToggledDistanceToCylinderAboveCupDynamic{R3,R2,S} <: PBDS.TaskMap{R3,R2,S}
    cup_rim_center::Vector{SVector{3,S}}
    axis::Vector{SVector{3,S}}
    radius::S
    cyl_height::S
    gap::S
    cup_velocity::Vector{S}
    scenario_state::Vector{Int}
end
function ToggledDistanceToCylinderAboveCupDynamic{R3,R2}(crc::Vector{SVector{3,S}}, a::Vector{SVector{3,S}}, 
        r::S, ch::S, g::S, cv::Vector{S}, ss) where {R3,R2,S}
    ToggledDistanceToCylinderAboveCupDynamic{R3,R2,S}(crc, [a[1]./norm(a[1])], r, ch, g, cv, ss)
end

function PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::ToggledDistanceToCylinderAboveCupDynamic{R3,R2})
    cup_rim_center, axis = task_map.cup_rim_center[1], task_map.axis[1]
    radius, cyl_height, gap = task_map.radius, task_map.cyl_height, task_map.gap
    
    α = 1.e-20                                  # Smoothing factor
    p_world = xme - cup_rim_center              # Position from origin in world frame
    y_dist_cyl = dot(axis, p_world)             # Axial distance from origin
    y_world = y_dist_cyl*axis                   # Axial position in world frame
    x_world = p_world - y_world                 # Transverse position in world frame
    x_dist_cyl = smooth_norm(x_world, α)        # Transverse distance from origin
    p_cyl = SA[x_dist_cyl, y_dist_cyl]

    # Switched to solid cylinder attractor
    p_cup_corner = SA[radius, 0.]
    p_cyl_corner1 = SA[radius, 100.]
    p_cyl_corner2 = SA[radius, gap+cyl_height]
    da = line_segment_distance(p_cyl, p_cup_corner, p_cyl_corner1) # Attractor
    dt = aligned_box_distance(p_cyl, p_cup_corner, p_cyl_corner2) # Toggle
    
    SA[da, dt]
end

In [10]:
function PBDS.potential_chart(xn, task::Attractor{<:ToggledDistanceToCylinderAboveCupDynamic{R3,R2,S}}, CN1::Chart{1,R2}) where S
    α = 1.e-3
    50*sqrt(xn[1]^2 + α)
end

PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:ToggledDistanceToCylinderAboveCupDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S  = 0*vn

PBDS.metric_chart(xn, task::Attractor{<:ToggledDistanceToCylinderAboveCupDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S = PBDS.default_metric(xn, task, CN)

function PBDS.weight_metric_chart(xn, vn, task::Attractor{<:ToggledDistanceToCylinderAboveCupDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S
    n = dim(R2)
    x_threshold = 0.02 # Threshold at which to toggle / untoggle state
    if xn[2] > x_threshold
        task.task_map.scenario_state[1] = 1
    elseif task.task_map.cup_velocity[1] < 0.02
        task.task_map.scenario_state[1] = 2
    end
    
    w = task.task_map.scenario_state[1] == 2 ? 1. : 0.
    W = SMatrix{n,n,eltype(xn)}(I)*w
end

### Toggled Attractor - Distance To Gap Cylinder Above Cup (State-toggled)

In [11]:
struct ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2,S} <: PBDS.TaskMap{R3,R2,S}
    cup_rim_center::Vector{SVector{3,S}}
    axis::Vector{SVector{3,S}}
    radius::S
    cyl_height::S
    gap::S
    cup_velocity::Vector{S}
    scenario_state::Vector{Int}
end
function ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2}(crc::Vector{SVector{3,S}}, a::Vector{SVector{3,S}}, r::S,
        ch::S, g::S, cv::Vector{S}, ss) where {R3,R2,S}
    ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2,S}(crc, [a[1]./norm(a[1])], r, ch, g, cv, ss)
end

function PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2})
    cup_rim_center, axis = task_map.cup_rim_center[1], task_map.axis[1]
    radius, cyl_height, gap = task_map.radius, task_map.cyl_height, task_map.gap
    
    α = 1.e-20                                  # Smoothing factor
    p_world = xme - cup_rim_center              # Position from origin in world frame
    y_dist_cyl = dot(axis, p_world)             # Axial distance from origin
    y_world = y_dist_cyl*axis                   # Axial position in world frame
    x_world = p_world - y_world                 # Transverse position in world frame
    x_dist_cyl = smooth_norm(x_world, α)        # Transverse distance from origin
    p_cyl = SA[x_dist_cyl, y_dist_cyl]

    # Solid cylinder attractor
    p_cup_corner = SA[radius, 0.]
    p_cyl_corner1 = SA[radius, gap]
    p_cyl_corner2 = SA[radius, gap + cyl_height]
    da = aligned_box_distance(p_cyl, p_cyl_corner1, p_cyl_corner2) # Attractor 
    dt = aligned_box_distance(p_cyl, p_cup_corner, p_cyl_corner2) # Toggle
    
    SA[da, dt]
end

In [12]:
function PBDS.potential_chart(xn, task::Attractor{<:ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2,S}}, CN1::Chart{1,R2}) where S
    α = 1.e-3
    30*sqrt(xn[1]^2 + α)
end

PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S  = 0*vn

PBDS.metric_chart(xn, task::Attractor{<:ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S = PBDS.default_metric(xn, task, CN)

function PBDS.weight_metric_chart(xn, vn, task::Attractor{<:ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S
    n = dim(R2)
    x_threshold = 0.02 # Threshold at which to toggle / untoggle state
    if xn[2] > x_threshold
        task.task_map.scenario_state[1] = 1
    elseif task.task_map.cup_velocity[1] < 0.02
        task.task_map.scenario_state[1] = 2
    end
    
    w = task.task_map.scenario_state[1] == 1 ? 1. : 0.
    W = SMatrix{n,n,eltype(xn)}(I)*w
end

### Toggled Attractor - Distance To Cup Rim

In [13]:
struct ToggledDistanceToCupRimDynamic{R3,R2,S} <: PBDS.TaskMap{R3,R2,S}
    cup_rim_center::Vector{SVector{3,S}}
    axis::Vector{SVector{3,S}}
    cyl_radius::S
    height::S
    gap::S
    cup_velocity::Vector{S}
    scenario_state::Vector{Int}
end
function ToggledDistanceToCupRimDynamic{R3,R2}(cb::Vector{SVector{3,S}}, a::Vector{SVector{3,S}}, cr::S,
        h::S, g::S, cv::Vector{S}, ss) where {R3,R2,S}
    ToggledDistanceToCupRimDynamic{R3,R2,S}(cb, [a[1]./norm(a[1])], cr, h, g, cv, ss)
end

function PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::ToggledDistanceToCupRimDynamic{R3,R2})
    cup_rim_center, axis = task_map.cup_rim_center[1], task_map.axis[1]
    cyl_radius, height, gap = task_map.cyl_radius, task_map.height, task_map.gap

    α = 1.e-20                                  # Smoothing factor
    p_world = xme - cup_rim_center              # Position from origin in world frame
    y_dist_cyl = dot(axis, p_world)             # Axial distance from origin
    y_world = y_dist_cyl*axis                   # Axial position in world frame
    x_world = p_world - y_world                 # Transverse position in world frame
    x_dist_cyl = smooth_norm(x_world, α)        # Transverse distance from origin
    p_cyl = SA[x_dist_cyl, y_dist_cyl]

    p_cup_corner = SA[cyl_radius, 0.]
    p_cyl_corner1 = SA[cyl_radius, gap]
    p_cyl_corner2 = SA[cyl_radius, gap + height]
    
    da = norm(p_cyl - p_cup_corner) # Attractor
    dt = aligned_box_distance(p_cyl, p_cup_corner, p_cyl_corner2, 2e-2) # Toggle
    
    SA[da, dt]
end

In [14]:
function PBDS.potential_chart(xn, task::Attractor{<:ToggledDistanceToCupRimDynamic{R3,R2,S}}, CN1::Chart{1,R2}) where S
    α = 1.e-2
    30*sqrt(xn[1]^2 + α)
end

PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:ToggledDistanceToCupRimDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S  = 0*vn

PBDS.metric_chart(xn, task::Attractor{<:ToggledDistanceToCupRimDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S = PBDS.default_metric(xn, task, CN)
function PBDS.weight_metric_chart(xn, vn, task::Attractor{<:ToggledDistanceToCupRimDynamic{R3,R2,S}}, CN::Chart{1,R2}) where S
    n = dim(R2)
    x_threshold = 0.02 # Threshold at which to toggle / untoggle state
    if xn[2] > x_threshold
        task.task_map.scenario_state[1] = 1
    elseif task.task_map.cup_velocity[1] < 0.02
        task.task_map.scenario_state[1] = 2
    end
    
    w = task.task_map.scenario_state[1] == 2 ? 1. : 0.
    W = SMatrix{n,n,eltype(xn)}(I)*w
end

### Joint Projection Task Map

In [15]:
struct HandJointProjection{R7,R1,S} <: PBDS.TaskMap{R7,R1,S} end
PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::HandJointProjection{R7,R1}) = SA[xme[7]]

### Projected Link Frame Transform

In [16]:
if !(@isdefined HandJointLinkFrameTransform)
    @computed struct HandJointLinkFrameTransform{R1,R12,S,JC} <: PBDS.TaskMap{R1,R12,S}
        state_cache::StateCache{S,JC}
        link_cache::fulltype(LinkFrameCache{7,S})
        joint_config::Vector{SVector{7,S}}
    end
end
HandJointLinkFrameTransform{R1,R12}(state_cache::StateCache{S,JC}, link_cache::LinkFrameCache{7,S},
        joint_config::Vector{SVector{7,S}}) where {n,R12,S,JC} =
    HandJointLinkFrameTransform{R1,R12,S,JC}(state_cache, link_cache, joint_config)
function PBDS.task_map_emb(::EmbRep, ::EmbRep, xme,
        task_map::HandJointLinkFrameTransform{R1,R12,S,JC}) where {n,S,JC}
    xme_full = SA[task_map.joint_config[1][1:6]..., xme[1]]
    PBDS.link_frame_transform(xme_full, task_map.state_cache, task_map.link_cache.link)
end

### Toggled Yaw Angle Attractor

In [17]:
struct ToggledHandYawAngleDynamic{R12,R2,S} <: PBDS.TaskMap{R12,R2,S}
    center_bottom::Vector{SVector{3,S}}
    axis::Vector{SVector{3,S}}
    cyl_radius::S
    height::S
    gap::S
end

function ToggledHandYawAngleDynamic{R12,R2}(cb::Vector{SVector{3,S}}, a::Vector{SVector{3,S}}, cr::S,
        h::S, g::S) where {R12,R2,S}
    ToggledHandYawAngleDynamic{R12,R2,S}(cb, [a[1]./norm(a[1])], cr, h, g)
end

function PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::ToggledHandYawAngleDynamic{R12,R2})
    center_bottom, axis = task_map.center_bottom[1], task_map.axis[1]
    cyl_radius, height, gap = task_map.cyl_radius, task_map.height, task_map.gap
    
    R = SMatrix{3,3,eltype(xme)}(xme[static(1):static(9)])
    position = xme[static(10):static(12)]
    align_axis = R*SA[0., 1., 0.]
    
    α = 1.e-6                                  # Smoothing factor
    p_world = position - center_bottom          # Position from origin in world frame
    y_dist_cyl = dot(axis, p_world)             # Axial distance from origin
    y_world = y_dist_cyl*axis                   # Axial position in world frame
    x_world = p_world - y_world                 # Transverse position in world frame
    x_dist_cyl = smooth_norm(x_world, α)        # Transverse distance from origin
    p_cyl = SA[x_dist_cyl, y_dist_cyl]
    
    des_axis = smooth_normalization(x_world, 1/α)
    
    d = smooth_abs(dot(align_axis, des_axis), α)
    da = d < 1. - 1e-6 ? acos(d) : 0.
    dt = norm(p_cyl) # Toggle
    
    SA[da, dt]
end

In [18]:
function PBDS.potential_chart(xn, task::Attractor{<:ToggledHandYawAngleDynamic{R12,R2,S}}, CN1::Chart{1,R2}) where S
    α = 1.e-3
    30*xn[1]^2
#     5*sqrt(xn[1]^2 + α)
end

PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:ToggledHandYawAngleDynamic{R12,R2,S}}, CN::Chart{1,R2}) where S  = 0*vn#-10*SA[0., vn[2]]

PBDS.metric_chart(xn, task::Attractor{<:ToggledHandYawAngleDynamic{R12,R2,S}}, CN::Chart{1,R2}) where S = PBDS.default_metric(xn, task, CN)
function PBDS.weight_metric_chart(xn, vn, task::Attractor{<:ToggledHandYawAngleDynamic{R12,R2,S}}, CN::Chart{1,R2}) where S
    n = dim(R2)
    xmin, xmax = 0.3, 0.1
    fmin, fmax = 0., 1.
    ε = 1.e-3
    w = logistic_val(xn[2], xmin, xmax, fmin, fmax, ε)
    W = SMatrix{n,n,eltype(xn)}(I)*w
end

### Toggled Hand Spin Axis Attractor

In [19]:
struct ToggledHandSpinAxisDynamic{R12,R2,S} <: PBDS.TaskMap{R12,R2,S}
    center_bottom::Vector{SVector{3,S}}
    axis::Vector{SVector{3,S}}
    cyl_radius::S
    height::S
    gap::S
end

function ToggledHandSpinAxisDynamic{R12,R2}(cb::Vector{SVector{3,S}}, a::Vector{SVector{3,S}}, cr::S,
        h::S, g::S) where {R12,R2,S}
    ToggledHandSpinAxisDynamic{R12,R2,S}(cb, [a[1]./norm(a[1])], cr, h, g)
end

function PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::ToggledHandSpinAxisDynamic{R12,R2})
    center_bottom, axis = task_map.center_bottom[1], task_map.axis[1]
    cyl_radius, height, gap = task_map.cyl_radius, task_map.height, task_map.gap
    
    R = SMatrix{3,3,eltype(xme)}(xme[static(1):static(9)])
    position = xme[static(10):static(12)] # Note that this is for the hand link, not the hand center
    align_axis = R*SA[0., 0., -1.]
    
    α = 1.e-6                                   # Smoothing factor
    p_world = position - center_bottom          # Position from origin in world frame
    y_dist_cyl = dot(axis, p_world)             # Axial distance from origin
    y_world = y_dist_cyl*axis                   # Axial position in world frame
    x_world = p_world - y_world                 # Transverse position in world frame
    x_dist_cyl = smooth_norm(x_world, α)        # Transverse distance from origin
    p_cyl = SA[x_dist_cyl, y_dist_cyl]

    p_corner1 = SA[cyl_radius, 0.]
    p_corner2 = SA[cyl_radius, gap+height]
    
    d = dot(align_axis, axis)
    da = d < 1. - 1e-10 ? acos(d) : 0.
    dt = line_segment_distance(p_cyl, p_corner1, p_corner2)
    SA[da, dt]
end

In [20]:
function PBDS.potential_chart(xn, task::Attractor{<:ToggledHandSpinAxisDynamic{R12,R2,S}}, CN1::Chart{1,R2}) where S
    30*xn[1]^2
end

PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:ToggledHandSpinAxisDynamic{R12,R2,S}}, CN::Chart{1,R2}) where S  = 0*vn#-10*SA[0., vn[2]]

PBDS.metric_chart(xn, task::Attractor{<:ToggledHandSpinAxisDynamic{R12,R2,S}}, CN::Chart{1,R2}) where S = PBDS.default_metric(xn, task, CN)
function PBDS.weight_metric_chart(xn, vn, task::Attractor{<:ToggledHandSpinAxisDynamic{R12,R2,S}}, CN::Chart{1,R2}) where S
    n = dim(R2)
    xmin, xmax = 1., 0.1
    fmin, fmax = 0., 1.
    ε = 1.e-3
    w = logistic_val(xn[2], xmin, xmax, fmin, fmax, ε)
    W = SMatrix{n,n,eltype(xn)}(I)*w
end

## Get Collision Spheres

In [21]:
urdf = joinpath("..", "..", "urdf", "panda", "table_panda_arm_hand_fingers_fixed.urdf")
mechanism = parse_urdf(Float64, urdf)

urdf_col = joinpath("..", "..", "urdf", "panda", "sphere_col", "table_panda_arm_hand_fingers_fixed.urdf")
sphere_ves = visual_elements(mechanism, URDFVisuals(urdf_col, tag="collision"))

state = MechanismState(mechanism)
state_cache = StateCache(mechanism)

n_links = 11
ind_sphere_link = [3:10; 12:14]
frame_list = [bodies(mechanism)[3].frame_definitions[1].from;
    [joints(mechanism)[i].frame_after for i in [3:9; 11:13]]]
sphere_links = bodies(mechanism)[ind_sphere_link]

link_caches = [LinkFrameCache(link, state) for link in sphere_links]
point_caches = [[] for i=1:n_links]
sphere_radii = [Float64[] for i=1:n_links]
sphere_centers = [[] for i=1:n_links]

for sphere_ve in sphere_ves
    i_parent = findfirst(isequal(sphere_ve.frame), frame_list)
    push!(sphere_radii[i_parent], sphere_ve.geometry.r)
    push!(sphere_centers[i_parent], sphere_ve.transform.translation)
    push!(point_caches[i_parent], FramePointCache(link_caches[i_parent], sphere_centers[i_parent][end]))
end

# Generate self-collision matrix
collision_matrix = zeros(n_links,n_links)
collision_matrix[6,1:2] .= 1
collision_matrix[7:11,1:3] .= 1
collision_matrix

bodies(mechanism)

15-element Vector{RigidBody{Float64}}:
 RigidBody: "world"
 RigidBody: "table"
 RigidBody: "panda_link0"
 RigidBody: "panda_link1"
 RigidBody: "panda_link2"
 RigidBody: "panda_link3"
 RigidBody: "panda_link4"
 RigidBody: "panda_link5"
 RigidBody: "panda_link6"
 RigidBody: "panda_link7"
 RigidBody: "panda_link8"
 RigidBody: "panda_hand"
 RigidBody: "panda_leftfinger"
 RigidBody: "panda_rightfinger"
 RigidBody: "panda_hand_center"

### Updatable mug struct

In [22]:
struct MugArmParams{S}
    arm_joint_config::Vector{SVector{7,S}}
    
    cup_center_bottom::Vector{SVector{3,S}}
    cup_rim_center::Vector{SVector{3,S}}
    cup_axis::Vector{SVector{3,S}}
    cyl_center_bottom::Vector{SVector{3,S}}
    cup_velocity::Vector{S}
    scenario_state::Vector{Int}
    
    cup_height::S
    gap::S
end

## Mug placement scenarios

In [23]:
in2m = 0.0254

# Scenario 1
f_cup_center_bottom(t) = SA[0.5, 0.5, 0.]
Time = 4.

# Scenario 2
f_cup_center_bottom(t) = SA[0.1, 0.2, 0.]
Time = 7.

# Scenario 3
function f_cup_center_bottom(t)
    ts = SA[0.5, 5.]
    p1 = SA[0.1, 0.2, 0.]
    p1dot = 0.1*SA[1., -0.5, 0.]
    p2 = SA[0.5, 0.55, 0.]
    
    ε = 1.e-2
    tmin, tmax = 0.5, 5.
    smin, smax = ts[1], ts[2]
    s = logistic_val(t, tmin, tmax, smin, smax, ε)
    p = spline2(s, p1, p1dot, p2, ts)
end
Time = 10.

# Scenario 4
function f_cup_center_bottom(t)
    ts = SA[0.05, 5.]
    p1 = SA[0.9, 0.35, 2*5*in2m]
    p1dot = 0.1*SA[-1.3, 0.5, 0.7]
    p2 = SA[0.45, 0.5, 0.]
    
    ε = 1.e-2
    tmin, tmax = 0.5, 5.
    smin, smax = ts[1], ts[2]
    s = logistic_val(t, tmin, tmax, smin, smax, ε)
    p = spline2(s, p1, p1dot, p2, ts)
end
Time = 10.

# Scenario 5
function f_cup_center_bottom(t)
    ts = SA[0.1, 5.]
    p1 = SA[0.9, 0.9, 0.]
    p1dot = 0.1*SA[-1., -1., 3.]
    p2 = SA[0.1, 0.2, 0.]
    
    ε = 1.e-2
    tmin, tmax = 0.3, 7.
    smin, smax = ts[1], ts[2]
    s = logistic_val(t, tmin, tmax, smin, smax, ε)
    p = spline2(s, p1, p1dot, p2, ts)
end
Time = 15.


function spline2(t, p1, p1dot, p2, ts)
    A = [ts[1]^2 ts[1] 1.
         2ts[1]  1.    0.
         ts[2]^2 ts[2] 1.]
    ax = A \ [p1[1], p1dot[1], p2[1]]
    ay = A \ [p1[2], p1dot[2], p2[2]]
    az = A \ [p1[3], p1dot[3], p2[3]]
    
    p = SA[ax[1]*t^2 + ax[2]*t + ax[3];
           ay[1]*t^2 + ay[2]*t + ay[3];
           az[1]*t^2 + az[2]*t + az[3]]
end

function f_cup_axis(t)
    SA[0.,0.,1.]
end

f_cup_axis (generic function with 1 method)

In [24]:
# Version with updateable mug
function PBDS.task_acceleration(xm, vm, t, node::TreeNode{M}, CM::Chart{I,M}, 
        state::MechanismState, cache::ControllerCache, mugparams::MugArmParams, robot_coord_rep=ChartRep();
        log_tasks=false) where {M<:PBDS.Manifold,I}
    set_configuration!(state, xm)
    setdirty!(cache)
    update_mug_params!(mugparams, xm, t)
    task_acceleration(xm, vm, node, CM, robot_coord_rep; log_tasks)
end

function update_mug_params!(mp::MugArmParams, xm, t)
    mp.arm_joint_config[1] = xm
    mp.cup_center_bottom[1] = f_cup_center_bottom(t)
    mp.cup_axis[1] = f_cup_axis(t)
    mp.cup_rim_center[1] = mp.cup_center_bottom[1] + mp.cup_height*mp.cup_axis[1]
    mp.cyl_center_bottom[1] = mp.cup_rim_center[1] + mp.gap*mp.cup_axis[1]
    cup_velocity_vec = ForwardDiff.jacobian(tvec -> f_cup_center_bottom(tvec[1]), SA[t])
    mp.cup_velocity[1] = norm(cup_velocity_vec)
    if cup_velocity[1] > 0.01
        mp.scenario_state[1] = 1
    end
    nothing
end

update_mug_params! (generic function with 1 method)

### Assemble tree

In [25]:
# Robot state
CM = Chart{1,R7}()
root = TreeNode(R7())

# Damping
add_child!(root, Damping(Identity{R7,R7,Float64}())) # 42 us, 40 alloc

# Joint Limits
hi = [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]
lo = [-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973]
for i = 1:7
    add_child!(root, JointLimit(Coordinate{R7,R1,Float64}(i),hi[i],lo[i])) # 1 us, 21 alloc
end

# Self-collision Constraints # 2610 us, 2841 alloc
for i = 1:n_links, j = 1:n_links
    if collision_matrix[i,j] == 1
        for ki = 1:length(sphere_radii[i]), kj = 1:length(sphere_radii[j])
            add_child!(root, SelfAvoidance(LinkSpherePairDistance{R7,R1}(state, state_cache,
                point_caches[i][ki], point_caches[j][kj], sphere_radii[i][ki], sphere_radii[j][kj])))
        end
    end
end

# Mug Avoidance
N = R1
CN = Chart{1,N}()
# cup_center_bottom = SA[0.55, 0.55, 0.]
cup_center_bottom = SA[1.0, 0.5, 0.]
cup_axis = SA[0., 0., 1.]
cup_axis = cup_axis ./ norm(cup_axis)
cup_radius = 0.04
cup_height = 0.105

v_cup_center_bottom = [cup_center_bottom]
v_cup_axis = [cup_axis]

# Box avoidance
in2m = 0.0254
ft2m = 0.3048
blackcolor = MeshBasicMaterial(color=RGBA(0.,0.,0.,0.5))
bluecolor = MeshBasicMaterial(color=RGBA(0.,0.,1.,0.3))
# Small box obstacles
dim_small = SA[5.25, 10.5, 5]*in2m
centers_small, rotations_small, gbrects_small, gtrects_small, colors_small = [], [], [], [], []
# Large box obstacles
dim_large = SA[10.75, 10.5, 5]*in2m
centers_large, rotations_large, gbrects_large, gtrects_large, colors_large = [], [], [], [], []

# Small boxes
push!(centers_small, SA[0.2, 0.5, dim_small[3]/2])
push!(rotations_small, RotZ(pi/4))
push!(colors_small, blackcolor)

push!(centers_small, SA[0.2, 0.5, 3*dim_small[3]/2])
push!(rotations_small, RotZ(-pi/8))
push!(colors_small, bluecolor)

push!(centers_small, SA[0.5, 0.2, dim_small[2]/2])
push!(rotations_small, RotZ(-pi/10)*RotX(pi/2))
push!(colors_small, bluecolor)

push!(centers_small, SA[0.6, 0.7, dim_small[2]/2])
push!(rotations_small, RotZ(-pi/3)*RotX(pi/2))
push!(colors_small, blackcolor)

# Large boxes
push!(centers_large, SA[0.8, 0.4, dim_large[3]/2])
push!(rotations_large, RotZ(2*pi/10))
push!(colors_large, bluecolor)
push!(centers_large, SA[0.8, 0.4, 3*dim_large[3]/2])
push!(rotations_large, RotZ(pi/10))
push!(colors_large, blackcolor)

link_frame_nodes = []
point_position_nodes = []

for i = 1:n_links
    push!(link_frame_nodes, add_child!(root, LinkFrameTransform{R7,R12}(state_cache, link_caches[i])))
    for j = 1:length(sphere_radii[i])
        push!(point_position_nodes, add_child!(link_frame_nodes[i],
            LinkSpherePosition{R12,R3}(point_caches[i][j])))
        # Cup avoidance
        add_child!(point_position_nodes[end], CollisionAvoidance(DistanceSphereToCupDynamic{R3,R1}(v_cup_center_bottom, 
            v_cup_axis, cup_radius, Float64(sphere_radii[i][j]), cup_height))) # 2000 us, 770 alloc
        # Table avoidance
        add_child!(point_position_nodes[end], CollisionAvoidance(DistanceSphereToBoxDynamic{R3,R1}(Float64(sphere_radii[i][j]), [GeometryBasics.Rect(GBVec(0.,0.,-ft2m), GBVec(SA[9.,12.,1]*ft2m))], [RotX(0)])))
        # Box avoidance
        for k in 1:length(centers_small)
            corner_small = centers_small[k] - dim_small./2
            push!(gbrects_small, GeometryBasics.Rect(GBVec(corner_small), GBVec(dim_small)))
            push!(gtrects_small, GeometryTypes.HyperRectangle(GeometryTypes.Vec(corner_small...), GeometryTypes.Vec(dim_small...)))
            add_child!(point_position_nodes[end], CollisionAvoidance(DistanceSphereToBoxDynamic{R3,R1}(Float64(sphere_radii[i][j]), [gbrects_small[end]], [rotations_small[k]])))
        end
        
        for k in 1:length(centers_large)
            corner_large = centers_large[k] - dim_large./2
            push!(gbrects_large, GeometryBasics.Rect(GBVec(corner_large), GBVec(dim_large)))
            push!(gtrects_large, GeometryTypes.HyperRectangle(GeometryTypes.Vec(corner_large...), GeometryTypes.Vec(dim_large...)))
            add_child!(point_position_nodes[end], CollisionAvoidance(DistanceSphereToBoxDynamic{R3,R1}(Float64(sphere_radii[i][j]), [gbrects_large[end]], [rotations_large[k]])))
        end
    end
end

# Hand Center Position
link = findbody(mechanism, "panda_hand_center")
hand_center_position = add_child!(root, JointToLinkPosition{R7,R3}(state_cache, link))

scenario_state = [1]
cup_velocity = [norm(ForwardDiff.jacobian(tvec -> f_cup_center_bottom(tvec[1]), SA[0.]))]

# Attactor cylinder above mug (horizontal attraction)
cup_rim_center = cup_center_bottom + cup_height*cup_axis
gap = 0.3
cyl_height = 0.1
cyl_center_bottom = cup_rim_center + gap*cup_axis
v_cup_rim_center = [cup_rim_center]
v_cyl_center_bottom = [cyl_center_bottom]
add_child!(hand_center_position, Attractor(ToggledDistanceToCylinderAboveCupDynamic{R3,R2}(v_cup_rim_center, v_cup_axis, cup_radius, cyl_height, gap, cup_velocity, scenario_state)))

# Attractor cylinder with gap above mug
add_child!(hand_center_position, Attractor(ToggledDistanceToGapCylinderAboveCupDynamic{R3,R2}(v_cup_rim_center, v_cup_axis, cup_radius, cyl_height, gap, cup_velocity, scenario_state)))

# Toggled attractor cylinder at mug rim
add_child!(hand_center_position, Attractor(ToggledDistanceToCupRimDynamic{R3,R2}(v_cup_rim_center, v_cup_axis, cup_radius, cyl_height, gap, cup_velocity, scenario_state)))

# Toggled yaw angle attractor
arm_joint_config = [@SVector zeros(7)]
child1 = add_child!(root, HandJointProjection{R7,R1,Float64}())
child2 = add_child!(child1, HandJointLinkFrameTransform{R1,R12}(state_cache, link_caches[9], arm_joint_config))
add_child!(child2, Attractor(ToggledHandYawAngleDynamic{R12,R2}(v_cup_rim_center, v_cup_axis, cup_radius, cyl_height, gap)))

# Toggled hand spin axis attractor
add_child!(link_frame_nodes[9], Attractor(ToggledHandSpinAxisDynamic{R12,R2}(v_cup_rim_center, v_cup_axis, cup_radius, cyl_height, gap)))

# End effector position damping
add_child!(hand_center_position, Damping(Identity{R3,R3,Float64}()))

# End effector S2 damping around cup
CN = Chart{SterProjNorth,S2}()
add_child!(hand_center_position, Damping(AngularPositionAroundPoint{R3,S2}(cup_rim_center)); CN)

mugparams = MugArmParams(arm_joint_config, v_cup_center_bottom, v_cup_rim_center, v_cup_axis, v_cyl_center_bottom, cup_velocity, scenario_state, cup_height, gap)

MugArmParams{Float64}(SVector{7, Float64}[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], SVector{3, Float64}[[1.0, 0.5, 0.0]], SVector{3, Float64}[[1.0, 0.5, 0.105]], SVector{3, Float64}[[0.0, 0.0, 1.0]], SVector{3, Float64}[[1.0, 0.5, 0.40499999999999997]], [0.003514036976636706], [1], 0.105, 0.3)

### Collect set of caches to update

In [26]:
cache = ControllerCache(link_caches, reduce(vcat, point_caches));

In [27]:
CM = Chart{1,R7}()

hi = [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]
lo = [-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973]

xm = SA[((hi+lo)/2)...]
Δx = SA[0., 0., 0., 0., 0., 0., 0.]
xm = xm + Δx
vm = @SVector zeros(7)

σxddot = task_acceleration(xm, vm, 0., root, CM, state, cache, mugparams)

7-element SVector{7, Float64} with indices SOneTo(7):
 -4.004114916706162
  5.227014610075148
 -4.004114916731965
 -0.43795242001539236
 -1.1184746228183762
 -1.2827745372663366
 -1.622002930198231e-19

In [28]:
@btime task_acceleration($xm, $vm, $root, $CM, $state, $cache, $mugparams)

  1.850 ms (17954 allocations: 1.17 MiB)


7-element SVector{7, Float64} with indices SOneTo(7):
 -4.004114916706162
  5.227014610075148
 -4.004114916731965
 -0.43795242001539236
 -1.1184746228183762
 -1.2827745372663366
 -1.622002930198231e-19

## Single Trajectory Test

In [29]:
# Time = 15.
dt = 0.02
t0 = 0.

time_dep = true
robot_coord_rep = EmbRep()
traj = propagate_tasks(xm, vm, root, CM, Time, dt, robot_coord_rep, state, cache, mugparams; time_dep, log_tasks=true)
traj.xm[end]

7-element SVector{7, Float64} with indices SOneTo(7):
  0.43689273354501057
  0.39238591305614884
  0.3789474696789413
 -2.3015134034483866
 -0.2948540208001395
  2.6467763415625156
  0.34487635450886506

In [30]:
σxddot = task_acceleration(traj.xm[end], traj.vm[end], 0., root, CM, state, cache, mugparams);

### Note: Correctly visualizing the arm requires the following:

1) Installation of the franka_ros ROS package https://github.com/frankaemika/franka_ros

2) Installation of RigidBodyDynamics v2.2.1 to the PBDS Julia environment. This version is currently only available in the master branch of the RigidBodyDynamics repo and can be installed through the Julia package manager via the following:
    
    activate PBDS
    add RigidBodyDynamics#master

In [31]:
# Modification to allow visualization of meshes
function MeshCatMechanisms.setelement!(mvis::MechanismVisualizer, frame::CartesianFrame3D, geometry::MeshCatMechanisms.MeshFile, material::AbstractMaterial, name::AbstractString="<element>")
    ext = lowercase(splitext(geometry.filename)[2])
    if ext == ".dae"
        obj = MeshFileObject(geometry.filename)
    else
        obj = Object(MeshFileGeometry(geometry.filename), material)
    end
    setelement!(mvis, frame, obj, name)
end

urdf_vis = joinpath("..", "..", "urdf", "panda", "table_mug_panda_arm_hand_fingers_fixed.urdf")
# urdf_vis = joinpath("..", "..", "urdf", "panda", "sphere_vis", "table_mug_panda_arm_hand_fingers_fixed.urdf") # To visualize collision spheres
mechanism_vis = parse_urdf(Float64, urdf_vis)
mvis = MechanismVisualizer(mechanism_vis, URDFVisuals(urdf_vis))
open(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8700
└ @ MeshCat /home/bylard/.julia/packages/MeshCat/X2AUA/src/visualizer.jl:73


Process(`xdg-open http://localhost:8700`, ProcessExited(0))

In [32]:
# Put mug in position
joint = findjoint(mechanism_vis, "mug_joint")
q = configuration(mvis.state, joint)
q[5:7] = f_cup_center_bottom(7)
set_configuration!(mvis.state, joint, q)
MeshCatMechanisms._render_state!(mvis)

# Visualize attractor cylinders
# mvis[:goal]
# setobject!(mvis[:goal][:cup], 
# Object(GeometryTypes.Cylinder(GeometryTypes.Point3(f_cup_center_bottom(0)...), GeometryTypes.Point3(f_cup_center_bottom(0) + cup_axis*cyl_height...), cup_radius),
#         MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

# setobject!(mvis[:goal][:cyl1], 
# Object(GeometryTypes.Cylinder(GeometryTypes.Point3((f_cup_center_bottom(0) + cup_axis*(cup_height + gap))...), GeometryTypes.Point3((f_cup_center_bottom(0) + cup_axis*(cup_height + gap +cyl_height))...), cup_radius),
#         MeshBasicMaterial(color=RGBA(0.,0.,1.,0.3))))

# Visualize box obstacles
mvis[:obs][:small]
for i = 1:length(centers_small)
    obj = mvis[:obs][:small][Symbol("box",i)]
    setobject!(obj, Object(gtrects_small[i], colors_small[i]))
    t = Translation(centers_small[i])
    R = LinearMap(rotations_small[i])
    settransform!(obj, t∘R∘inv(t))
end

mvis[:obs][:large]
for i = 1:length(centers_large)
    obj = mvis[:obs][:large][Symbol("box",i)]
    setobject!(obj, Object(gtrects_large[i], colors_large[i]))
    t = Translation(centers_large[i])
    R = LinearMap(rotations_large[i])
    settransform!(obj, t∘R∘inv(t))
end

trans = Translation(-0.5,-0.3,0.)
rot1 = LinearMap(RotY(0.0)*RotX(-π/2))
settransform!(mvis[""], trans ∘ rot1)
rot2 = LinearMap(RotY(0.0)*RotZ(2.3))
settransform!(mvis["/Cameras/default"], rot2)
setprop!(mvis["/Cameras/default/rotated/<object>"], "zoom", 3.0)

MeshCat Visualizer with path /Cameras/default/rotated/<object> at http://localhost:8700

Opening in existing browser session.


In [33]:
q_traj = [SA[1., 0., 0., 0., f_cup_center_bottom(dt*(i-1))..., traj.xm[i]...] for i in 1:length(traj.xm)]
t_traj = collect(range(0., length=length(q_traj), stop=Time));

In [36]:
MeshCatMechanisms.animate(mvis, t_traj, q_traj)

In [ ]:
animation = Animation(mvis, t_traj, q_traj)
setanimation!(mvis, animation)